Setup Library 

In [ ]:
import tensorflow_datasets as tfds 
# Download the Dataset Oford iit Pets 
dataset, info = tfds.load("oxford_iiit_pet:3.*.*", with_info=True)

Prepare the Datasets

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from keras import backend 

# Define parameters
image_size = 512 
# Mean 
mean = tf.constant([0.485 , 0.456 , 0.407])
# Standard deviation 
std = tf.constant([0.029 , 0.0224 , 0.225])

# Thiết lập phương thức bình thường hóa hình ảnh 
def normalize(input_image , input_mask):
    # Chuyển đổi hình ảnh thành dạng tensor float 32
    input_image = tf.image.convert_image_dtype(input_image , tf.float32)
    # Thực hiện tiêu chuẩn hình ảnh  = image - mean / maximum(std, backend.epsilon())
    input_image = (input_image - mean) / tf.maximum(std, backend.epsilon()) 
    # Input_mask -= 1
    input_mask -= 1 
    return input_image , input_image

# Thiết lập phương thưcs tải dữ liệu 
# Với chức năng đặt lại kích thước cho hình ảnh 
# và kích thước input_mask
def load_image(datapoint):
    input_image = tf.image.resize(datapoint["image"], (image_size , image_size))
    # Resize input_mask sử dụng phương pháp upsamling với bilinear 
    input_mask = tf.image.resize(
        datapoint["segmentation_mask"],
        (image_size, image_size),
        method="bilinear",
    )

    # Thực hiện bình thường hóa dữ liệu 
    # với input_image , input_mask 
    input_image , input_mask = normalize(input_image, input_mask)
    # Chuyển vị hình ảnh để có được tensor với kích thước [channel , h , w]
    # vì đây là định dạng trong pytorch sử dụng để lư trữ ảnh 
    input_image = tf.transpose(input_image , (2 , 0, 1))
    # Trả về input _image  và labels dưới dạng từ điển 
    return {"pixel_values": input_image, "labels": tf.squeeze(input_mask)}


In [ ]:
# Sử dụng các tiện ích để chuản bị các đối tượng dữ liệu Dataset bao gồm việc 
# Tìm nạp trước cho hiệu Suất . 
# Thay đổi các batch_size cho phù hợp với kích thước của bộ nhớ GPU được sử dụng 
# để đào tạo 
auto = tf.data.AUTOTUNE
batch_size = 4 

# Thiết lập bộ dữ liệu đào tạo 
train_ds = (
    # ĐẦU TIÊN là dữ liệu nguồn
    dataset["Train"]
    # Lưu chúng vào bộ nhớ cache 
    .cache()
    # Xáo trộn dữ liệu với mộ bộ = batch_size * 4
    .shuffle(batch_size * 4)
    # sử dụng hàm map để biến đổi dữ liệu 
    # Với hàm laod_image để chuẩn hóa và biến đổi các hình ảnh 
    # Lấy song song các bộ dự liệu đặt = auto để tự động điều chỉnh cho 
    # phù hợp với mô hình 
    .map(load_image , numm_parallel_calls=auto)
    # chia thành các batch mỗi batch = batch_size 
    .batch(batch_size)
    # Tìm nạp trước các bộ dữ liệu để có được trạng thái sẵn sàng cho việc 
    # sử dụng dữ liệu 
    .prefetch(auto)
)


# Thiết lập bộ dữ liệu huấn luyện
test_ds = (
    # ĐẦU TIÊN là dữ liệu nguồn
    dataset["test"]
    # sử dụng hàm map để biến đổi dữ liệu 
    # Với hàm laod_image để chuẩn hóa và biến đổi các hình ảnh 
    # Lấy song song các bộ dự liệu đặt = auto để tự động điều chỉnh cho 
    # phù hợp với mô hình 
    .map(load_image , numm_parallel_calls=auto)
    # chia thành các batch mỗi batch = batch_size 
    .batch(batch_size)
    # Tìm nạp trước các bộ dữ liệu để có được trạng thái sẵn sàng cho việc 
    # sử dụng dữ liệu 
    .prefetch(auto)
)
"""Kiểm tra hình dạng của những hình ảnh đầu vào và phân đoạn đồ thị của chúng"""
train_ds.elements_spec 

Visualize Dataset

In [ ]:
# Thực hiện trực quan hóa dữ liệu 
# Xây dựng phương thức hiển thị dữ liệu 
import matplotlib.pyplot as plt 
def display (display_list):
    # Định cấu hình khung hiển thị inch 
    plt.figure(figsize=(15 , 15))

    # Đặt tiêu đề hiển thị với nhãn là Input Image , True Mask ,Predict Mask 
    title = ["Input Image", "True Mask","Predicted_mask"]

    # Duyệt qua danh sách các phần tử của display_list 
    for i in range(len(display_list)):
        # Biều dạng subplot với 1 hình ảnh mỗi hàng 
        plt.subplot(1 , len(display_list) , i+1)
        # Add tiêu đề 
        plt.title(title[i])
        # Hiển thị hình ảnh với Image show cần chuyển đổi từ ma trận biểu diễn 
        # sang hình ảnh với utils.array_to_image 
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        # Bỏ đi các đường kẻ trục 
        plt.axis("off")
    # Hiển thị hàng loạt với plt.show 
    plt.show()

# Duyệt qua 1 danh sách với 2 mẫu từ bộ dữ liệu đào tạo 
for sample in train_ds.take(2):

    # Tạo 2 biến sample image và sample mask là hình ảnh và nhãn tương ứng của ảnh 
    sample_image , sample_mask = sample["pixel_values"][0], sample["labels"][0]
    # Chuyển vị hình ảnh để có dạng image_size , image_size , channels 
    sample_image = tf.transpose(sample_image, (1 ,2 , 0))
    # Thêm 1 chiều vào vị trí cuối cùng cho nhãn
    sample_mask = tf.expand_dims(sample_mask , -1)
    # Trả về sample và nhãn tương ứng 
    display([sample_image , sample_mask])


Load a pretrained SegFormer checkpoint

In [ ]:
from transformers import TFSegformerForSemanticSegmentation 

# Tạo model_checkpoint 
model_checkpoint = "nvidia/mit-b0"
# nhãn cho mô hình là một từ điển với key và values tương ứng 
id2label = {0: "outer", 1: "inner", 2: "border"}
# Duyệt qua danh sách nhãn và lấy ra các bộ tương ứng 
label2id = {label: id for id, label in id2label.items()}
num_labels = len(id2label) # lấy ra số lượng nhãn cho mô hình đào tạo trước 
# Đào tạo trước mô hình đồng thời thêm vào các tham số tương ứng 
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,

)

Compile the Model 

In [ ]:
# Khởi tạo tham số lr 
lr = 0.00006
# Trình tối ưu hóa  = Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# Trìnj biên dịch cho mô hình 
model.compile(optimizer=optimizer)

Prediction callback to monitor training progress

In [ ]:
from IPython.display import clear_output 

# Xây dựng phương thức tạo mặt nạ 
def create_mask(pred_mask):
    # lấy ra chỉ số lớn nhất theo chỉ số hàng 
    # tức là mỗi vector sẽ lấy ra chỉ số lớn nhất 
    pred_mask = tf.math.argmax(pred_mask, axis=1)
    # Thêm 1 chiều cuối vào tensor_mask  => shape = [bach_size , h , w , -1]
    pred_mask = tf.expand_dims(pred_mask, -1)
    # Trả về phần từ đầu tiên của tensor tức là mặt nạ của ảnh đầu tiên trong batch 
    return pred_mask[0]


# Xây dựng phương thức hiển thị mặt nạ dự đoán 
def show_predictions(dataset = None , num=1):
    # Kiểm tra xem có tồn tại Dataset 
    if dataset: 
        # Duyệt qua danh sách dataset với số lượng num được lấy 
        for sample in dataset.take(num):
            # lấy ra ảnh và mặt nạ 
            images , masks = sample["pixel_values"] , sample["labels"]
            # Thêm chiều cuối cùng cho mask => shape = [batch_size , h , w , 1]
            masks = tf.expand_dims(masks , -1)
            # Thực hiện dự đoán mask cho hình ảnh 
            pred_masks = model.predict(images).logits
            # Reshape hình ảnh với kich [batch_size ,channel, h , w] -> 
            # [batch_size , h , w , channels] vì ta chuyển từ thư viện pytorch sang tensorflow 
            # nên cần thay đổi lại định dạng khác 
            images = tf.transpose(images , (0 , 2 , 3 , 1))
            # Hiển thị hình ảnh và mặt nạ tương ứng của nó với mặt nạ dự đoán 
            # hiển thị bộ ảnh đầu tiên tronh danh sách 
            display([images[0], masks[0], create_mask(pred_masks)])
    # Trường hợp còn lại
    else:
        # Thực hiện hiển thị 
        display(
            [
                # với hình ảnh và mặt nạ labels
                sample_image,
                sample_mask,
                # Tạo mặt nạ dự đoán 
                create_mask(model.predict(tf.expand_dims(sample_image, 0))),
            ]
        )



# xây dựng phương thưcs callback 
class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset , **kwargs):
        super().__init__(**kwargs)
        self.dataset = dataset 
    
    def on_epoch_end(self, epoch, logs=None):
        # xóa đầu ra 
        clear_output(wait=True)
        # Thực hiện hiển thị ra các dự đoán từ nguồn ảnh 
        show_predictions(self.dataset)
        print("\nSample Prediction after epoch {}\n".format(epoch + 1))


Train the model

In [ ]:
# Increase the number of epochs if the results are not of expected quality.
epochs = 5

history = model.fit(
    train_ds,
    validation_data=test_ds,
    callbacks=[DisplayCallback(test_ds)],
    epochs=epochs,
)

show_predictions(test_ds, 5)